In [17]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [18]:
driver = webdriver.Chrome()

In [19]:
def intitialize_program(driver):
    driver.get('https://safer.fmcsa.dot.gov/CompanySnapshot.aspx')
    #getting the MC/MX ID MCQ
    goto_mc_mx = driver.find_element(By.ID, '2')
    goto_mc_mx.click()

In [20]:
def record_check(page_code):
    souping = BeautifulSoup(page_code, 'html.parser')
    itag = souping.find('i')
    if itag is not None:
        txtitag = itag.text
        if txtitag == "Record Not Found":
            return False
        return True
#Submit the form
def submit_form_range(counter,driver):
    _range = driver.find_element(By.ID, '4')
    _range.clear()
    _range.send_keys(counter)
    #selecting the search button
    searchBtn = driver.find_element(By.XPATH, '/html/body/form/p/table/tbody/tr[4]/td/input')
    searchBtn.click()
    page_code = driver.page_source
    exists=record_check(page_code)
    #problem in the code was that if a record existed then we would get an error with it as selenium could not find its tag
    # ---->Solution Brainstorming: Use BS to parse the html website 
    # ----> Check if "Record Not Found" if yes return true else true 
    if exists:
        print('The record exists')
        return first_section_dict(driver) 
        # its true that the record exists
    else:
        #its false that the record does not exists # so we go back # make and start with a new entry 
        driver.back()
        return False

In [21]:
#this will then be converted into a function and will return a dictionary of all the entries in the list
#i want these queries till the Operations Function
def first_section_dict(driver):
    queries_tags = []
    entry_tags = []
    queries = driver.find_elements(By.CLASS_NAME, 'querylabelbkg')
    queries_entries = driver.find_elements(By.CLASS_NAME, 'queryfield')
    
    for query,entry in zip(queries,queries_entries):
        if query.text == "Operation Classification:":
            op_block = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table')
            break
        queries_tags.append(query)
        entry_tags.append(entry)
    
    #now we know that the entry in the 15th tag is not right (in entry) so we are going to manually replace it same for 13th index
#     mileage_per_year_tag = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[12]/td[2]')
#     entry_tags[15] = mileage_per_year_tag
    #so we need to upade the driver and then update the Date
    #--> update 14 'MCS-150 Form Date:'
    #--> update 13 ''Drivers:'
    #--> update 
    driver_no = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[11]/td[2]/font/b')
    date = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[12]/td[1]')
    entry_tags[13] = driver_no
    entry_tags[14] = date

    firstSectionDict = {}
    for query,entry in zip(queries_tags,entry_tags):
        firstSectionDict[query.text] = entry.text
        print(query.text,  entry.text)
        
    return firstSectionDict

In [22]:
def CheckCarrierBlock(carrier_op_dict):
    if carrier_op_dict['Interstate'] == 'X':
        return True

def CarrierBlock(driver):
    carrier_op_block = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]')
    interstate = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td[2]/font')
    interstate_Qfield = driver.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td[1]')

    intrastate_hm = driver.find_element(By.XPATH,'/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[2]/font')
    intrastate_hmQfield = driver.find_element(By.XPATH,'/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[1]')

    intrastate_nhm = driver.find_element(By.XPATH,'/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[3]/table/tbody/tr[2]/td[2]/font')
    intrastate_nhmQfield = driver.find_element(By.XPATH,'/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[16]/td/table/tbody/tr[2]/td[3]/table/tbody/tr[2]/td[1]')

    carrier_op_dict = {interstate.text:interstate_Qfield.text,intrastate_hm.text:intrastate_hmQfield.text,intrastate_nhm.text:intrastate_nhmQfield.text}
    return carrier_op_dict

In [23]:
#checking if the record full fills the conditions and how the modularized code is going to run
#the main code that would run all 
def checkingFirstSection(first_section_dict):
    ctr = 1
    for key, value in first_section_dict.items():
        if (key == 'Entity Type:' and value.strip() == 'Carrier'):
            #means that the first_section_dict Key, Entity type is '''Carrier'''.
            ctr += 1
        if (key == 'Operating Status:' and value.strip() == 'AUTHORIZED FOR Property'):
            #means that the first_section_dict Key, Entity type is '''AUTHORIZED FOR Property'''.
            ctr += 1
        if ctr == 2:
            print(ctr)
            return True
        
        else:
            return False

#<---------------------------MAIN------------------------------->
# range_start = int(input("From what range do you want to start?"))
# range_end = int(input("Which range do you want to end?"))
range_diff = 850000- 800000
intitialize_program(driver)
MX_MC_lst = []
for i in range(1420000,1420500):
    mx_mc = i
    if submit_form_range(mx_mc,driver) == False:
        driver.back()
        continue 
    else:
        first_section_dict = submit_form_range(mx_mc,driver)
        print(first_section_dict)
        check_dict_bool = checkingFirstSection(first_section_dict)
        print(check_dict_bool)
        #"'''check_dict_bool'''" this will check if the dictionary is correct record and a match
        check_carr = CheckCarrierBlock(CarrierBlock(driver))
        print(check_carr)
        #'''check_carr''' this will give us the t/f if its interstate or no
        
    if check_dict_bool and check_carr:
        print(check_dict_bool)
        print(check_carr) 
        MX_MC_lst.append(mx_mc)
        driver.back()
        continue
    else:
        driver.back()


The record exists
Entity Type: CARRIER  
Operating Status: AUTHORIZED FOR Property
Out of Service Date: None
Legal Name: LEICK TRUCKING LLC 
DBA Name:  
Physical Address: 1315 MYRA AVE
JANESVILLE, WI   53548  
Phone: (608) 449-4852  
Mailing Address: 1315 MYRA AVE
JANESVILLE, WI   53548-0140  
USDOT Number: 3875174 
State Carrier ID Number:  
MC/MX/FF Number(s): MC-1420000
 
DUNS Number: -- 
Power Units: 2 
Drivers: 2 
MCS-150 Form Date: 04/13/2023 
MCS-150 Mileage (Year): 


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/form/p/table/tbody/tr[4]/td/input"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71D128EF2+54786]
	(No symbol) [0x00007FF71D095612]
	(No symbol) [0x00007FF71CF4A64B]
	(No symbol) [0x00007FF71CF8B79C]
	(No symbol) [0x00007FF71CF8B91C]
	(No symbol) [0x00007FF71CFC6D87]
	(No symbol) [0x00007FF71CFABEAF]
	(No symbol) [0x00007FF71CFC4D02]
	(No symbol) [0x00007FF71CFABC43]
	(No symbol) [0x00007FF71CF80941]
	(No symbol) [0x00007FF71CF81B84]
	GetHandleVerifier [0x00007FF71D477F52+3524194]
	GetHandleVerifier [0x00007FF71D4CD800+3874576]
	GetHandleVerifier [0x00007FF71D4C5D7F+3843215]
	GetHandleVerifier [0x00007FF71D1C5086+694166]
	(No symbol) [0x00007FF71D0A0A88]
	(No symbol) [0x00007FF71D09CA94]
	(No 

In [15]:
#op_block --> this is the block with the Operation Classification
# op_block.text
# first_table_Xpath= "/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table/tbody/tr[2]/td[1]" #first table
# sec_table_Xpath= '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table/tbody/tr[2]/td[2]' #second
# third_table_Xpath= "/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table/tbody/tr[2]/td[3]" #third

# #this is the code for table one  
# # /html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td[1]
# check = op_block.find_element(By.XPATH, '/html/body/p/table/tbody/tr[2]/td/table/tbody/tr[2]/td/center[1]/table/tbody/tr[14]/td/table/tbody/tr[2]/td[1]/table/tbody/tr[2]/td[1]')
# #first tr always gives us the row index of the first column in the Table1
# check.text

# lst = [1420000, 1420002, 1420004, 1420006, 1420009, 1420012, 1420014, 1420016, 1420017, 1420019, 1420020, 1420031, 1420039, 1420052, 1420054, 1420056, 1420061, 1420070, 1420071, 1420072, 1420078, 1420091, 1420092, 1420101, 1420103, 1420105, 1420113, 1420115, 1420120, 1420124, 1420126, 1420128, 1420129, 1420132, 1420133, 1420136, 1420139, 1420146, 1420149, 1420151, 1420157, 1420159, 1420160, 1420162, 1420164, 1420168, 1420173, 1420180, 1420181, 1420184, 1420185, 1420188, 1420189, 1420191, 1420196]
# df = pd.DataFrame(lst,columns = ['MX/MC'])
# df.to_csv('C:\\Users\\mhass\\Desktop\\3rd Semester\\Programming with AI\\CheckingMSAFER', index = False)
